In [73]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

import tqdm

In [74]:
dataset = pd.read_csv('final_dataset.csv')
dataset = dataset.set_index('Unnamed: 0')

In [75]:
drop_columns = ['eda_time', 'time', 'gender']
dataset.drop(drop_columns, axis=1, inplace=True)

In [76]:
# dataset.sort_values(by='participantID', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)

,perc_change,abs_change,perc_bpm,abs_bpm,perc_ibi,abs_ibi,perc_sdnn,abs_sdnn,perc_sdsd,abs_sdsd,...,participantID,change_in_stressed,change_in_amused,change_in_nervous,change_in_engaged,change_in_lost,change_in_disappointed,change_in_excited,change_in_frustrated,change_in_contempt
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2,-18.175823,-0.037491,1.199979,15.775700,0.833348,-126.753156,1.153764,20.215637,1.230926,20.593818,...,0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
14,5.554695,0.148284,0.938055,-5.642874,1.066036,43.494591,0.599577,-68.969036,0.432693,-90.102220,...,1,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,-11.869750,-0.113862,1.297776,16.061588,0.770549,-255.236951,3.011952,103.539287,2.329031,48.048323,...,2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,38.392876,0.101000,0.604905,-49.595285,1.653152,312.195617,0.871641,-15.329947,0.737058,-23.484337,...,3,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,1.0,0.0
13,-16.508557,-0.190186,0.870330,-13.412365,1.148990,86.425781,1.940864,28.880547,2.632402,52.550976,...,3,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
10,10.470302,0.044198,1.057743,3.851429,0.945409,-49.107143,0.605240,-27.159178,0.735194,-8.439078,...,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
16,33.068751,0.053382,1.817241,52.742973,0.550285,-418.094758,1.539984,56.441399,3.717112,97.221522,...,3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,57.804164,0.093312,1.088640,6.188703,0.918577,-69.972826,1.894265,39.745241,1.235645,7.147833,...,4,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
15,9.757678,0.495065,0.902527,-9.357401,1.108000,67.500000,1.068665,8.960377,0.986647,-1.815082,...,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [107]:
# perc_columns = [col for col in dataset.columns if 'perc' in col]
# dataset_perc = pd.DataFrame(dataset, columns=perc_columns)
# dataset_perc['participantID'] = dataset['participantID'].values

dataset_perc = dataset[dataset.columns.drop(list(dataset.filter(regex='abs')))]

In [119]:
dataset_perc.columns[14]

'participantID'

In [124]:
# x_features = total_train_data.columns[0:28]
# y_labels = total_train_data.columns[29:]

x_features = [col for col in dataset.columns if 'perc' in col]
y_labels = dataset_perc.columns[15:]

### Machine Learning Models

In [128]:
# Support Vecotr Machine

def svc(x_train,x_test, y_train, y_test):
    # Create the RFE object and compute a cross-validated score.
    svc = SVC(kernel="linear")
    # The "accuracy" scoring shows the proportion of correct classifications

    min_features_to_select = 1  # Minimum number of features to consider
    rfecv = RFECV(
        estimator=svc,
        step=1,
        # cv=StratifiedKFold(3),
        scoring="accuracy",
        min_features_to_select=min_features_to_select,
    )
    rfecv.fit(x_train, y_train)
    rfecv.predict(x_test)

    # print()
    # print("The train accuracy:", rfecv.score(x_train, y_train))
    # print("The test accuracy:", rfecv.score(x_test, y_test))
    # print("Optimal number of features : %d" % rfecv.n_features_)
    # print(rfecv.transform(x_train))
    print('Selected Features:', x_features[rfecv.get_support(indices=False)])

    return x_features[rfecv.get_support(indices=False)], rfecv.score(x_test, y_test)

In [112]:
# K- Nearst Neighbours

def KNN(y_train, N):
    # Create the RFE object and compute a cross-validated score.
    knn = KNeighborsClassifier(n_neighbors = N)
    # train the model
    knn.fit(x_train, y_train)
    scores = cross_val_score(knn, x_train, y_train, cv=2, scoring='accuracy')
    accuracy = scores.mean()

    return accuracy, knn

In [113]:
# Decision Trees

def decision_tree(x_test, y_train, y_test):
    # Create the RFE object and compute a cross-validated score.
    dt = DecisionTreeClassifier(random_state=0)
    # The "accuracy" scoring shows the proportion of correct classifications

    min_features_to_select = 1  # Minimum number of features to consider
    rfecv = RFECV(
        estimator=dt,
        step=1,
        cv=StratifiedKFold(3),
        scoring="accuracy",
        min_features_to_select=min_features_to_select,
    )
    rfecv.fit(x_train, y_train)
    rfecv.predict(x_test)

    # print()
    # print("The train accuracy:", rfecv.score(x_train, y_train))
    # print("The test accuracy:", rfecv.score(x_test, y_test))
    # print("Optimal number of features : %d" % rfecv.n_features_)
    # print(rfecv.transform(x_train))
    print('Selected Features:', x_features[rfecv.get_support(indices=False)])

    return x_features[rfecv.get_support(indices=False)], rfecv.score(x_test, y_test)

### Model training

In [130]:
# SVM training

selected_features = []
accuracy_score = []
for j in range(len(y_labels)):
    print(y_labels[j])
    try:
    
        participantIDs = np.arange(11)

        for i, a in enumerate(participantIDs):
            train_ids = participantIDs[participantIDs!=i]
            test_ids = participantIDs[participantIDs==i]
            total_train_data = dataset_perc.loc[dataset_perc['participantID'].isin(train_ids)]
            test_data = dataset_perc.loc[dataset_perc['participantID'].isin(test_ids)]
            x_features = total_train_data.columns[0:14]
            y_labels = total_train_data.columns[15:]  

            x_train = total_train_data[x_features]
            x_test = test_data[x_features] 

            y_train = total_train_data[y_labels[j]]
            y_test = test_data[y_labels[j]]

            s_features, test_score = svc(x_train, x_test, y_train, y_test)

            selected_features.append(s_features)
            accuracy_score.append(test_score)
            print(test_score)
            print()
            print()
            print('-'*15)
    except Exception:
        print()
        print()
        print('-'*15)
        continue







change_in_stressed
Selected Features: Index(['perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_pnn50', 'perc_hr_mad',
       'perc_sd1/sd2', 'perc_breathingrate'],
      dtype='object')


---------------
Selected Features: Index(['perc_ibi', 'perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_pnn50',
       'perc_hr_mad', 'perc_sd1', 'perc_sd2', 'perc_s', 'perc_sd1/sd2',
       'perc_breathingrate'],
      dtype='object')


---------------
Selected Features: Index(['perc_sdsd', 'perc_rmssd', 'perc_pnn50', 'perc_hr_mad', 'perc_sd1',
       'perc_sd1/sd2'],
      dtype='object')


---------------
Selected Features: Index(['perc_ibi'], dtype='object')


---------------
Selected Features: Index(['perc_sdsd', 'perc_rmssd', 'perc_hr_mad', 'perc_sd1', 'perc_s',
       'perc_sd1/sd2'],
      dtype='object')


---------------
Selected Features: Index(['perc_sdsd', 'perc_rmssd', 'perc_pnn50', 'perc_hr_mad', 'perc_sd1',
       'perc_sd1/sd2'],
      dtype='object')



C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(



---------------
Selected Features: Index(['perc_ibi', 'perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_hr_mad',
       'perc_sd1/sd2'],
      dtype='object')


---------------
Selected Features: Index(['perc_sdsd'], dtype='object')


---------------
Selected Features: Index(['perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_hr_mad', 'perc_sd1',
       'perc_sd1/sd2'],
      dtype='object')


---------------
Selected Features: Index(['perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_hr_mad', 'perc_sd1',
       'perc_s', 'perc_sd1/sd2'],
      dtype='object')


---------------
Selected Features: Index(['perc_ibi', 'perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_pnn50',
       'perc_hr_mad', 'perc_sd1', 'perc_sd2', 'perc_s', 'perc_sd1/sd2',
       'perc_breathingrate'],
      dtype='object')


---------------
change_in_amused
Selected Features: Index(['perc_ibi', 'perc_breathingrate'], dtype='object')


---------------
Selected Features: Index(['perc_bpm', 'perc_ibi', 'perc_breathingrate'], dtype='ob

C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Index(['perc_bpm', 'perc_ibi'], dtype='object')


---------------
Selected Features: Index(['perc_change', 'perc_ibi', 'perc_sdnn', 'perc_pnn20', 'perc_pnn50',
       'perc_hr_mad', 'perc_sd2', 'perc_s', 'perc_sd1/sd2',
       'perc_breathingrate'],
      dtype='object')


---------------
Selected Features: Index(['perc_ibi'], dtype='object')


---------------
Selected Features: Index(['perc_ibi'], dtype='object')


---------------
Selected Features: Index(['perc_ibi'], dtype='object')


---------------
Selected Features: Index(['perc_bpm', 'perc_ibi', 'perc_pnn20', 'perc_pnn50', 'perc_sd2',
       'perc_s', 'perc_sd1/sd2', 'perc_breathingrate'],
      dtype='object')


---------------
Selected Features: Index(['perc_pnn50'], dtype='object')


---------------
Selected Features: Index(['perc_ibi'], dtype='object')


---------------
change_in_nervous
Selected Features: Index(['perc_ibi'], dtype='object')


---------------


...

C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Selected Features: Index(['perc_pnn20'], dtype='object')


---------------
Selected Features: Index(['perc_pnn20'], dtype='object')


---------------
Selected Features: Index(['perc_pnn20'], dtype='object')



C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(



---------------
Selected Features: Index(['perc_pnn20'], dtype='object')


---------------


C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
# KNN training

for i in range(len(y_labels)):

    error = np.inf
    for k in range(2, 5):
        y_train = total_train_data[y_labels[i]]
        y_test = test_data[y_labels[i]]
        accuracy, knn = KNN(y_train, k)
        if accuracy < error:
            error = accuracy

    print(y_labels[i],"--->", "accuracy:", knn.score(x_test, y_test), 'K=', k)
    print()
    print()
    print('-'*15)
        

In [ ]:
# Decision tree training

selected_features = []
accuracy_score = []
for i in range(len(y_labels)):
    try:
        print('try', y_labels[i])
        y_train = total_train_data[y_labels[i]]
        y_test = test_data[y_labels[i]]
        s_features, test_score = decision_tree(x_test, y_train, y_test, )

        selected_features.append(s_features)
        accuracy_score.append(test_score)
        print()
        print()
        print('-'*15)
    except Exception:
        print()
        print()
        print('error occurence')
        print('-'*15)
        continue

In [7]:
# test participant ID --- 0,1,2,5
train_participantID = [3, 4, 6, 7, 8, 9, 10]
test_participantID = [0,1,2,5]
total_train_data = dataset.loc[dataset['participantID'].isin(train_participantID)]
test_data = dataset.loc[dataset['participantID'].isin(test_participantID)]

In [8]:
x_features = total_train_data.columns[0:28]
y_labels = total_train_data.columns[29:]

In [9]:
x_train = total_train_data[x_features]
x_test = test_data[x_features]

In [14]:
# total_train_data[y_labels]

total_train_data[x_features]

,perc_change,abs_change,perc_bpm,abs_bpm,perc_ibi,abs_ibi,perc_sdnn,abs_sdnn,perc_sdsd,abs_sdsd,...,perc_sd1,abs_sd1,perc_sd2,abs_sd2,perc_s,abs_s,perc_sd1/sd2,abs_sd1/sd2,perc_breathingrate,abs_breathingrate
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,19.174717,0.137370,1.077506,6.277595,0.928069,-53.285256,0.810544,-16.755300,0.805243,-10.059671,...,0.665924,-20.374796,0.698089,-33.467917,0.464874,-11365.889660,0.953924,-0.025350,1.601164,0.104098
1,-36.539961,-0.584716,0.491861,-71.571070,2.033094,440.084586,3.821121,98.920209,2.669653,33.959400,...,1.821205,25.167794,4.993054,128.319381,9.093375,25041.492751,0.364748,-0.605833,0.333471,-0.164901
5,-33.060336,-0.099749,1.053153,4.444379,0.949530,-36.216518,3.188974,78.884350,3.451631,65.005112,...,3.230195,70.581457,3.044893,80.271564,9.835601,34484.561055,1.060857,0.049064,0.752137,-0.051845
6,-12.953205,-0.062564,1.004254,0.368691,0.995764,-2.932692,0.897959,-14.100424,1.395917,31.543606,...,0.894707,-16.104477,0.896088,-11.227664,0.801736,-10293.503746,0.998459,-0.002181,0.783226,-0.048210
7,57.804164,0.093312,1.088640,6.188703,0.918577,-69.972826,1.894265,39.745241,1.235645,7.147833,...,0.995993,-0.142086,2.239747,59.100417,2.230772,6535.731704,0.444690,-0.413036,0.520637,-0.101474
8,38.392876,0.101000,0.604905,-49.595285,1.653152,312.195617,0.871641,-15.329947,0.737058,-23.484337,...,0.652849,-39.301406,0.655016,-44.005098,0.427626,-25967.047935,0.996691,-0.002937,0.760101,-0.057083
9,24.695338,1.036414,0.763320,-23.888605,1.310067,184.322667,1.211367,28.488133,1.228246,24.212915,...,0.897981,-13.709367,1.855571,104.412116,1.666268,34326.714184,0.483938,-0.568258,0.589256,-0.094279
10,10.470302,0.044198,1.057743,3.851429,0.945409,-49.107143,0.605240,-27.159178,0.735194,-8.439078,...,0.740322,-8.114932,0.531546,-43.229017,0.393515,-5494.498968,1.392772,0.133009,1.057558,0.009146
11,-0.558582,-0.001281,0.995048,-0.393157,1.004977,3.761574,0.434070,-59.252333,0.570460,-26.189162,...,0.609031,-29.595789,0.341045,-83.144896,0.207707,-23774.030684,1.785778,0.471420,1.119238,0.023387


In [70]:
def svc(x_train,x_test, y_train, y_test):
    # Create the RFE object and compute a cross-validated score.
    svc = SVC(kernel="linear")
    # The "accuracy" scoring shows the proportion of correct classifications

    min_features_to_select = 1  # Minimum number of features to consider
    rfecv = RFECV(
        estimator=svc,
        step=1,
        cv=StratifiedKFold(3),
        scoring="accuracy",
        min_features_to_select=min_features_to_select,
    )
    rfecv.fit(x_train, y_train)
    rfecv.predict(x_test)

    print()
    print("The train accuracy:", rfecv.score(x_train, y_train))
    print("The test accuracy:", rfecv.score(x_test, y_test))
    print("Optimal number of features : %d" % rfecv.n_features_)
    # print(rfecv.transform(x_train))
    print('Selected Features:', x_features[rfecv.get_support(indices=False)])

    return x_features[rfecv.get_support(indices=False)], rfecv.score(x_test, y_test)

In [72]:
# selected_features = []
# accuracy_score = []
# for i in range(len(y_labels)):
#     # if i == 4 or i == 5:
#     #     continue
#    # try:
#     print('try', y_labels[i])
    
#     participantIDs = np.arange(11)

#     for i, a in enumerate(participantIDs):
#         train_ids = participantIDs[participantIDs!=i]
#         test_ids = participantIDs[participantIDs==i]
#         total_train_data = dataset.loc[dataset['participantID'].isin(train_ids)]
#         test_data = dataset.loc[dataset['participantID'].isin(test_ids)]
#         x_features = total_train_data.columns[0:28]
#         y_labels = total_train_data.columns[29:]  

#         x_train = total_train_data[x_features]
#         x_test = test_data[x_features] 

#         y_train = total_train_data[y_labels]
#         y_test = test_data[y_labels]

#         s_features, test_score = svc(x_train,x_test, y_train, y_test)

#     selected_features.append(s_features)
#     accuracy_score.append(test_score)
#     print()
#     print()
#     print('-'*15)
#    # except Exception:
#     #    print()
#      #   print()
#       #  print('-'*15)
#        # continue

try change_in_stressed


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multiclass-multioutput' instead.

In [18]:
# # SVM
# selected_features = []
# accuracy_score = []
# for i in range(len(y_labels)):
#     # if i == 4 or i == 5:
#     #     continue
#     try:
#         print('try', y_labels[i])
#         y_train = total_train_data[y_labels[i]]
#         y_test = test_data[y_labels[i]]
#         s_features, test_score = svc(x_test, y_train, y_test)

#         selected_features.append(s_features)
#         accuracy_score.append(test_score)
#         print()
#         print()
#         print('-'*15)
#     except Exception:
#         print()
#         print()
#         print('-'*15)
#         continue


try change_in_stressed

The train accuracy: 1.0
The test accuracy: 0.25
Optimal number of features : 3
Selected Features: Index(['perc_change', 'abs_sdnn', 'abs_sdsd'], dtype='object')


---------------
try change_in_amused

The train accuracy: 0.5384615384615384
The test accuracy: 0.75
Optimal number of features : 1
Selected Features: Index(['perc_pnn20'], dtype='object')


---------------
try change_in_nervous


C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(



The train accuracy: 1.0
The test accuracy: 0.75
Optimal number of features : 7
Selected Features: Index(['perc_change', 'abs_bpm', 'abs_ibi', 'abs_sdnn', 'abs_sdsd',
       'abs_rmssd', 'abs_hr_mad'],
      dtype='object')


---------------
try change_in_engaged


C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



The train accuracy: 0.9230769230769231
The test accuracy: 0.25
Optimal number of features : 3
Selected Features: Index(['perc_change', 'abs_rmssd', 'abs_hr_mad'], dtype='object')


---------------
try change_in_lost


---------------
try change_in_disappointed


...


The train accuracy: 0.7692307692307693
The test accuracy: 1.0
Optimal number of features : 3
Selected Features: Index(['abs_change', 'abs_rmssd', 'perc_s'], dtype='object')


---------------
try change_in_excited


---------------
try change_in_frustrated


C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



The train accuracy: 0.6923076923076923
The test accuracy: 0.5
Optimal number of features : 1
Selected Features: Index(['abs_hr_mad'], dtype='object')


---------------
try change_in_contempt


C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



The train accuracy: 0.7692307692307693
The test accuracy: 0.75
Optimal number of features : 1
Selected Features: Index(['perc_s'], dtype='object')


---------------


In [64]:
def KNN(y_train, N):
    # Create the RFE object and compute a cross-validated score.
    knn = KNeighborsClassifier(n_neighbors = N)
    # train the model
    knn.fit(x_train, y_train)
    scores = cross_val_score(knn, x_train, y_train, cv=2, scoring='accuracy')
    accuracy = scores.mean()

    return accuracy, knn



In [68]:
# # KNN
# import numpy as np

# for i in range(len(y_labels)):

#     error = np.inf
#     for k in range(2, 5):
#         y_train = total_train_data[y_labels[i]]
#         y_test = test_data[y_labels[i]]
#         accuracy, knn = KNN(y_train, k)
#         if accuracy < error:
#             error = accuracy

#     print(y_labels[i],"--->", "accuracy:", knn.score(x_test, y_test), 'K=', k)
#     print()
#     print()
#     print('-'*15)
        



change_in_stressed ---> accuracy: 0.25 K= 4


---------------
change_in_amused ---> accuracy: 0.25 K= 4


---------------
change_in_nervous ---> accuracy: 1.0 K= 4


---------------
change_in_engaged ---> accuracy: 0.25 K= 4


---------------
change_in_lost ---> accuracy: 1.0 K= 4


---------------
change_in_disappointed ---> accuracy: 1.0 K= 4


---------------
change_in_excited ---> accuracy: 0.25 K= 4


---------------
change_in_frustrated ---> accuracy: 0.5 K= 4


---------------
change_in_contempt ---> accuracy: 0.75 K= 4


---------------


C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\junegoo94\miniconda3\envs\comp0084\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has